In [ ]:
# Install the required libraries
!pip install openai pinecone-client


In [ ]:
import os
import openai
import pinecone
from pinecone import Pinecone, ServerlessSpec

# API keys and environment variables
OPENAI_API_KEY = 'open ai key'
PINECONE_API_KEY = 'pinecone api'
PINECONE_ENVIRONMENT = 'us-east-1'
PINECONE_INDEX_NAME = 'chat-history' #you can name anything based on your preference
MAX_TOKENS = 225

# Initialize OpenAI client
openai.api_key = OPENAI_API_KEY

# Initialize Pinecone client
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
existing_indexes = pinecone_client.list_indexes()
if PINECONE_INDEX_NAME not in existing_indexes:
    print(f"Creating index '{PINECONE_INDEX_NAME}'...")
    pinecone_client.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,  # Adjust dimension based on model
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region=PINECONE_ENVIRONMENT)
    )
else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists, connecting to the index.")

# Connect to the index
index = pinecone_client.Index(PINECONE_INDEX_NAME)


In [ ]:
# Define the conversation history with each message approximately 20 tokens long
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    # Additional conversation data...
]


In [ ]:
# Function to get embeddings using OpenAI API
def get_embedding(text):
    response = openai.Embedding.create(input=text, model="text-embedding-ada-002")
    return response['data'][0]['embedding']

# Define the function to add embeddings to Pinecone
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    """
    Encodes each message in the history using OpenAI and upserts into Pinecone.
    """
    # Connect to the index
    index = pinecone_client.Index(index_name)

    # Loop to encode the history and upsert into Pinecone
    for idx, message in enumerate(history):
        embedding = get_embedding(message)
        index.upsert([(str(idx), embedding)])


In [ ]:
# Define the RAG mechanism to retrieve relevant history
def retrieve_relevant_history(query, index_name='chat-history'):
    """
    Encodes the query and retrieves relevant historical messages from Pinecone.
    """
    index = pinecone_client.Index(index_name)
    query_embedding = get_embedding(query)
    result = index.query(queries=[query_embedding], top_k=5)  # Retrieves top 5 relevant messages
    return [match['id'] for match in result['matches']]


In [ ]:
# Function to prepare the prompt
def prepare_prompt(test_prompt, history, index_name='chat-history'):
    """
    Retrieves relevant history using the RAG mechanism, combines it with the test prompt,
    and ensures the prompt doesn't exceed the token limit.
    """
    relevant_message_ids = retrieve_relevant_history(test_prompt, index_name)
    relevant_history = [history[int(msg_id)] for msg_id in relevant_message_ids]
    
    combined_prompt = test_prompt + "\n\n" + "\n".join(relevant_history)
    if len(combined_prompt.split()) > MAX_TOKENS:
        combined_prompt = " ".join(combined_prompt.split()[:MAX_TOKENS])
    return combined_prompt, relevant_history


In [ ]:
def test_final_prompt():
    """
    This function:
    1. Defines the final test prompt.
    2. Prepares the prompt using the prepare_prompt function.
    3. Calls OpenAI to generate a response.
    4. Prints the final test prompt, context, and response.
    """
    final_test_prompt = "Do you think it will help me stay fit?"
    prepared_prompt, context_referred = prepare_prompt(final_test_prompt, history)
    
    # Call OpenAI API to generate a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        prompt=prepared_prompt,
        max_tokens=100
    )
    
    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Referred: {context_referred}")
    print(f"Final Test Prompt Response: {response.choices[0].text.strip()}")

# Call the test function to generate the Final Test Prompt Response
test_final_prompt()
